# New address parser

In [1]:

entities =[{'type': 'unit_id',
    'text': ' 36 - 40',
    'start': 5,
    'end': 13,
    'confidence': 0.9996221},
{'type': 'number_filter',
    'text': 'even',
    'start': 15,
    'end': 19,
    'confidence': 0.999858},
{'type': 'unit_id',
    'text': ' 42-44',
    'start': 21,
    'end': 27,
    'confidence': 0.9986492},
{'type': 'building_name',
    'text': ' climb house',
    'start': 28,
    'end': 40,
    'confidence': 0.999928},
{'type': 'unit_id',
    'text': ' 1-5',
    'start': 47,
    'end': 51,
    'confidence': 0.997581},
{'type': 'building_name',
    'text': ' down buildings',
    'start': 51,
    'end': 66,
    'confidence': 0.99611604},
{'type': 'street_name',
    'text': ' chapel street',
    'start': 67,
    'end': 81,
    'confidence': 0.99992263},
{'type': 'postcode',
    'text': ' se45 6pq',
    'start': 90,
    'end': 99,
    'confidence': 0.99991864}]


In [ ]:
import pandas as pd
from typing import List, Dict, Any, Optional
from enhance_ocod.labelling.ner_regex import xx_to_yy_regex
from enhance_ocod.address_parsing import expand_dataframe_numbers, AddressGraph, process_addresses


In [ ]:


# Test
def test_with_visualization():
    entities = [
        {'type': 'unit_type', 'text': 'Flat', 'start': 0, 'end': 4, 'confidence': 0.999},
        {'type': 'unit_id', 'text': ' 1A', 'start': 4, 'end': 7, 'confidence': 0.999},
        {'type': 'unit_type', 'text': ' Flat', 'start': 8, 'end': 12, 'confidence': 0.999},
        {'type': 'unit_id', 'text': ' 1B', 'start': 12, 'end': 15, 'confidence': 0.999},
        {'type': 'street_number', 'text': ' 25', 'start': 16, 'end': 19, 'confidence': 0.999},
        {'type': 'street_name', 'text': ' Oak Street', 'start': 19, 'end': 30, 'confidence': 0.999},
        {'type': 'city', 'text': ' London', 'start': 31, 'end': 38, 'confidence': 0.999}
    ]
    
    graph = AddressGraph(entities)
    graph.visualize_graph()
    result = graph.get_addresses()
    print("\nParsed Addresses:")
    print(result)



test_with_visualization()

graph = AddressGraph(entities)
graph.visualize_graph()
result = graph.get_addresses()
print("\nParsed Addresses:")
print(result)


Address Graph:
city: 'London'
  street_name: 'Oak Street'
    street_number: '25'
      unit_id: '1A' (unit_type: Flat)
      unit_id: '1B' (unit_type:  Flat)

Leaf nodes: [('unit_id', '1A'), ('unit_id', '1B')]

Parsed Addresses:
  unit_type unit_id street_number street_name    city
0      Flat      1A            25  Oak Street  London
1      Flat      1B            25  Oak Street  London
Address Graph:
postcode: 'se45 6pq'
  street_name: 'chapel street'
    building_name: 'climb house'
      number_filter: 'even'
        unit_id: '36 - 40'
      unit_id: '42-44'
    building_name: 'down buildings'
      unit_id: '1-5'

Leaf nodes: [('unit_id', '36 - 40'), ('unit_id', '42-44'), ('unit_id', '1-5')]

Parsed Addresses:
   unit_id number_filter   building_name    street_name  postcode
0  36 - 40          even     climb house  chapel street  se45 6pq
1    42-44           NaN     climb house  chapel street  se45 6pq
2      1-5           NaN  down buildings  chapel street  se45 6pq


In [3]:
temp_ents = {'summary': {'total_addresses': 1,
  'successful_parses': 1,
  'failed_parses': 0,
  'success_rate': 1.0,
  'batch_size_used': 512},
 'results': [{'row_index': 0,
   'datapoint_id': 0,
   'original_address': 'Flats 36 - 40 (even), 42-44, climb house, Flats 1-5 down buildings, chapel street, London, se45 6pq',
   'entities': [{'type': 'unit_id',
     'text': ' 36 - 40',
     'start': 5,
     'end': 13,
     'confidence': 0.9996221},
    {'type': 'number_filter',
     'text': 'even',
     'start': 15,
     'end': 19,
     'confidence': 0.999858},
    {'type': 'unit_id',
     'text': ' 42-44',
     'start': 21,
     'end': 27,
     'confidence': 0.9986492},
    {'type': 'building_name',
     'text': ' climb house',
     'start': 28,
     'end': 40,
     'confidence': 0.999928},
    {'type': 'unit_id',
     'text': ' 1-5',
     'start': 47,
     'end': 51,
     'confidence': 0.997581},
    {'type': 'building_name',
     'text': ' down buildings',
     'start': 51,
     'end': 66,
     'confidence': 0.99611604},
    {'type': 'street_name',
     'text': ' chapel street',
     'start': 67,
     'end': 81,
     'confidence': 0.99992263},
    {'type': 'postcode',
     'text': ' se45 6pq',
     'start': 90,
     'end': 99,
     'confidence': 0.99991864}],
   'parsed_components': {'unit_id': [' 36 - 40', ' 42-44', ' 1-5'],
    'number_filter': ['even'],
    'building_name': [' climb house', ' down buildings'],
    'street_name': [' chapel street'],
    'postcode': [' se45 6pq']}}]}

In [4]:
process_addresses(temp_ents['results'])


,unit_id,number_filter,building_name,street_name,postcode,datapoint_id
0,36 - 40,even,climb house,chapel street,se45 6pq,0
1,42-44,NaN,climb house,chapel street,se45 6pq,0
2,1-5,NaN,down buildings,chapel street,se45 6pq,0


# New pipeline approach

In [5]:
from enhance_ocod.address_parsing import (
    load_and_prep_OCOD_data,
    load_postcode_district_lookup,
)
from enhance_ocod.locate_and_classify import (
    load_voa_ratinglist
    counts_of_businesses_per_oa_lsoa,
    add_geographic_metadata,
    enhance_ocod_with_gazetteers,
    add_business_matches,
    property_class,
    tag_multi_property
)
from pathlib import Path
from tqdm import tqdm
import time
import gc  # Add for memory management
import pandas as pd
import numpy as np
import pickle
# There is a warning related to bfill and ffill which is basically internal to pandas so silencing here
import warnings



warnings.filterwarnings("ignore", message=".*Downcasting object dtype arrays.*")

SCRIPT_DIR = Path("/teamspace/studios/this_studio/enhance_ocod/notebooks")

# ====== CONSTANT PATHS AND SETTINGS ======
input_dir = SCRIPT_DIR.parent / "data" / "ocod_history"
output_dir = SCRIPT_DIR.parent / "data" / "ocod_history_processed2"
model_path = (
    SCRIPT_DIR.parent / "models" / "address_parser_original_fullset" / "final_model"
)


ONSPD_dir = SCRIPT_DIR.parent / "data" / "onspd"
ONSPD_files = list(ONSPD_dir.glob("*"))
ONSPD_path = ONSPD_files[0] if ONSPD_files else None

price_paid_dir = SCRIPT_DIR.parent / "data" / "price_paid_data"
price_paid_files = list(price_paid_dir.glob("*"))
price_paid_path = price_paid_files[0] if price_paid_files else None


voa_dir = SCRIPT_DIR.parent / "data" / "voa"
voa_files = list(voa_dir.glob("*"))
voa_path = voa_files[0] if voa_files else None


processed_price_paid_dir = SCRIPT_DIR.parent / "data" / "processed_price_paid"
output_dir.mkdir(parents=True, exist_ok=True)

parsed_results_dir = SCRIPT_DIR.parent / "data" / "parsed_ocod_dicts2"
parsed_results_dir.mkdir(parents=True, exist_ok=True)

# List of all zip files in input_dir
#
# TESTING!!! only 10 files!
#
all_files = sorted([f for f in input_dir.glob("OCOD_FULL_*.zip")])


print("Loading common reference data...")
postcode_district_lookup = load_postcode_district_lookup(str(ONSPD_path))
voa_businesses = load_voa_ratinglist(str(voa_path), postcode_district_lookup)


zip_file = Path('/teamspace/studios/this_studio/enhance_ocod/data/ocod_history/OCOD_FULL_2022_02.zip')

Loading common reference data...


/teamspace/studios/this_studio/enhance_ocod/src/enhance_ocod/address_parsing.py:34: DtypeWarning: Columns (18,31,39,44,52) have mixed types. Specify dtype option on import or set low_memory=False.
  postcode_district_lookup = pd.read_csv(f)[


Initial rows: 2293839


# Load results

In [6]:
out_name = zip_file.stem + ".parquet"
out_path = output_dir / out_name

# Define parsed results file path
parsed_results_file = parsed_results_dir / f"{zip_file.stem}_parsed_results.pkl"

print(f"Processing {zip_file.name}...")

# Load and process the OCOD data
ocod_data = load_and_prep_OCOD_data(str(zip_file))

###############
# Parse addresses
###############
if parsed_results_file.exists():
    print(f"Loading cached parsing results for {zip_file.name}...")
    with open(parsed_results_file, "rb") as f:
        results = pickle.load(f)
    print(
        f"Loaded cached results with success rate: {results['summary']['success_rate']:.1%}"
    )
else:
    from enhance_ocod.inference import parse_addresses_pipeline
    import torch
    torch.set_float32_matmul_precision("medium")
    print(f"Parsing addresses for {zip_file.name}...")
    start_time = time.time()

    results = parse_addresses_pipeline(
        df=ocod_data,
        short_batch_size=128,  # The default seems really slow, might be to do with loading not sure
        model_path=str(model_path),
        target_column="property_address",
    )

    end_time = time.time()
    print(f"Address parsing took {end_time - start_time:.2f} seconds")
    print(f"Success rate: {results['summary']['success_rate']:.1%}")

    # Save parsing results
    print(f"Saving parsing results to {parsed_results_file}...")
    with open(parsed_results_file, "wb") as f:
        pickle.dump(results, f)


Processing OCOD_FULL_2022_02.zip...
Loading cached parsing results for OCOD_FULL_2022_02.zip...
Loaded cached results with success rate: 100.0%


In [8]:
processed_addresses_df = process_addresses(results['results'])

post_processed_data = processed_addresses_df.merge(
    ocod_data, how="left", left_on="datapoint_id", right_index=True
)[
        [
            "title_number",
            "tenure",
            "unit_id",
            "unit_type",
            "number_filter",
            "building_name",
            "street_number",
            "street_name",
            "postcode",
            "city",
            "district",
            "county",
            "region",
            "price_paid",
            "property_address",
            "country_incorporated",
        ]
    ]

post_processed_data["postcode"] = post_processed_data["postcode"].str.upper()

post_processed_data

,title_number,tenure,unit_id,unit_type,number_filter,building_name,street_number,street_name,postcode,city,district,county,region,price_paid,property_address,country_incorporated
0,MAN296957,Leasehold,NaN,NaN,NaN,Westleigh Lodge Care Home,NaN,Nel Pan Lane,WN7 5JT,Leigh,WIGAN,GREATER MANCHESTER,NORTH WEST,NaN,"Westleigh Lodge Care Home, Nel Pan Lane, Leigh...",ISLE OF MAN
1,MAN297185,Leasehold,1,Flat,NaN,NaN,NaN,Canal Street,M1 3HE,Manchester,MANCHESTER,GREATER MANCHESTER,NORTH WEST,270000.0,"Flat 1, 1a Canal Street, Manchester (M1 3HE)",JERSEY
2,MAN299183,Leasehold,201,Flat,NaN,NaN,NaN,Regent Road,M3 4AY,Manchester,MANCHESTER,GREATER MANCHESTER,NORTH WEST,243516.0,"Flat 201, 1 Regent Road, Manchester (M3 4AY)",BRITISH VIRGIN ISLANDS
3,MAN300821,Freehold,NaN,NaN,NaN,NaN,NaN,Gerard Street,WN4 9AA,Wigan,WIGAN,GREATER MANCHESTER,NORTH WEST,NaN,"Land at 2a Gerard Street, Ashton In Makerfield...",DENMARK
4,MAN300971,Leasehold,111,Unit,NaN,Timber Wharf,NaN,Worsley Street,M15 4NZ,Manchester,MANCHESTER,GREATER MANCHESTER,NORTH WEST,342000.0,"Unit 111, Timber Wharf, Worsley Street, Manche...",JERSEY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119828,AGL470128,Leasehold,NaN,NaN,NaN,NaN,NaN,Lime Street,EC3M 7AW,London,CITY OF LONDON,GREATER LONDON,GREATER LONDON,NaN,"2nd Floor, 52 Lime Street, London (EC3M 7AW)",IRELAND
119829,AGL473126,Leasehold,NaN,NaN,NaN,NaN,155,Bishopsgate,EC2M 3AD,London,CITY OF LONDON,GREATER LONDON,GREATER LONDON,NaN,"155 Bishopsgate, London (EC2M 3AD)",JAPAN
119830,AGL473595,Leasehold,NaN,NaN,NaN,NaN,NaN,Lime Street,EC3M 7AF,London,CITY OF LONDON,GREATER LONDON,GREATER LONDON,NaN,"9th and 10th floors, 52 Lime Street, London (E...",AUSTRALIA
119831,AGL475468,Leasehold,NaN,NaN,NaN,NaN,NaN,Bevis Marks,EC3A 7BA,London,CITY OF LONDON,GREATER LONDON,GREATER LONDON,NaN,"part of Tenth Floor, 6 Bevis Marks, London (EC...",INDIA


In [10]:
pre_process_ocod = add_geographic_metadata(post_processed_data, postcode_district_lookup)

In [11]:
building_gazetteer = pd.read_parquet('...data/gazetteer/building_gazetteer.parquet')
building_gazetteer['fraction'] = 1
district_gazetteer = pd.read_parquet('.../data/gazetteer/district_gazetteer.parquet')
street_gazetteer = pd.read_parquet('.../data/gazetteer/street_gazetteer.parquet')

In [13]:
pre_process_ocod2 = pre_process_ocod
pre_process_ocod2['building_name'] = pre_process_ocod2['building_name'].str.lower()
pre_process_ocod2['street_name2'] = pre_process_ocod2['street_name2'].str.lower()
enhanced  =  enhance_ocod_with_gazetteers(pre_process_ocod2, building_gazetteer, district_gazetteer, street_gazetteer)



In [14]:
enhanced.loc[enhanced['lsoa11cd'].notna(), ['unit_id', 'unit_type', 'building_name', 'street_number', 'street_name',
       'postcode', 'city', 'district', 'county', 'region', 'price_paid',
       'property_address', 'geog_match', 'match_prob']]

,unit_id,unit_type,building_name,street_number,street_name,postcode,city,district,county,region,price_paid,property_address,geog_match,match_prob
0,NaN,NaN,westleigh lodge care home,NaN,Nel Pan Lane,WN7 5JT,Leigh,WIGAN,GREATER MANCHESTER,NORTH WEST,NaN,"Westleigh Lodge Care Home, Nel Pan Lane, Leigh...",NaN,NaN
1,1,Flat,NaN,NaN,Canal Street,M1 3HE,Manchester,MANCHESTER,GREATER MANCHESTER,NORTH WEST,270000.0,"Flat 1, 1a Canal Street, Manchester (M1 3HE)",NaN,NaN
2,201,Flat,NaN,NaN,Regent Road,M3 4AY,Manchester,MANCHESTER,GREATER MANCHESTER,NORTH WEST,243516.0,"Flat 201, 1 Regent Road, Manchester (M3 4AY)",NaN,NaN
3,NaN,NaN,NaN,NaN,Gerard Street,WN4 9AA,Wigan,WIGAN,GREATER MANCHESTER,NORTH WEST,NaN,"Land at 2a Gerard Street, Ashton In Makerfield...",NaN,NaN
4,111,Unit,timber wharf,NaN,Worsley Street,M15 4NZ,Manchester,MANCHESTER,GREATER MANCHESTER,NORTH WEST,342000.0,"Unit 111, Timber Wharf, Worsley Street, Manche...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119828,NaN,NaN,NaN,NaN,Lime Street,EC3M 7AW,London,CITY OF LONDON,GREATER LONDON,GREATER LONDON,NaN,"2nd Floor, 52 Lime Street, London (EC3M 7AW)",NaN,NaN
119829,NaN,NaN,NaN,155,Bishopsgate,EC2M 3AD,London,CITY OF LONDON,GREATER LONDON,GREATER LONDON,NaN,"155 Bishopsgate, London (EC2M 3AD)",NaN,NaN
119830,NaN,NaN,NaN,NaN,Lime Street,EC3M 7AF,London,CITY OF LONDON,GREATER LONDON,GREATER LONDON,NaN,"9th and 10th floors, 52 Lime Street, London (E...",NaN,NaN
119831,NaN,NaN,NaN,NaN,Bevis Marks,EC3A 7BA,London,CITY OF LONDON,GREATER LONDON,GREATER LONDON,NaN,"part of Tenth Floor, 6 Bevis Marks, London (EC...",NaN,NaN


In [15]:
enhanced.groupby('geog_match')['match_prob'].describe()

/tmp/ipykernel_8844/2654210722.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  enhanced.groupby('geog_match')['match_prob'].describe()


,count,mean,std,min,25%,50%,75%,max
geog_match,,,,,,,,
building,1162.0,1.000000,0.000000,1.00000,1.000000,1.0000,1.0,1.0
street,25004.0,0.696205,0.310967,0.02414,0.396552,0.7625,1.0,1.0


## Analysing the entries with no geo-match

In [16]:
# Get the missing LSOA records
missing_lsoa_addresses = enhanced.loc[enhanced['lsoa11cd'].isna(), 'property_address']

# Count how many contain each keyword
keyword_counts = {}
keywords = ['airspace', 'air space', 'land', 'plot', 'car park', 'carpark' ]

for keyword in keywords:
    count = missing_lsoa_addresses.str.contains(keyword, case=False, na=False).sum()
    keyword_counts[keyword] = count
    print(f"'{keyword}': {count}")

# If you want to see which addresses contain ANY of these keywords
any_keyword_mask = missing_lsoa_addresses.str.contains('|'.join(keywords), case=False, na=False)
any_keyword_count = any_keyword_mask.sum()
print(f"\nAddresses containing any of {keywords}: {any_keyword_count}")

# If you want to see the actual addresses that contain these keywords
addresses_with_keywords = missing_lsoa_addresses[any_keyword_mask]
print(f"\nSample addresses containing keywords:")
print(addresses_with_keywords.head(10).tolist())

# Summary counts
print(f"\nSummary:")
print(f"Total missing LSOA: {len(missing_lsoa_addresses)}")
print(f"Contains keywords: {any_keyword_count}")
print(f"Percentage: {any_keyword_count/len(missing_lsoa_addresses)*100:.1f}%")

'airspace': 42
'air space': 13
'land': 3474
'plot': 1256
'car park': 313
'carpark': 2

Addresses containing any of ['airspace', 'air space', 'land', 'plot', 'car park', 'carpark']: 4977

Sample addresses containing keywords:
['Land at Queensgate Centre, Peterborough', 'Land and Buildings on the North East Side of Surbiton Street, Sheffield', 'Land on the north west side of New Drove, North Brink, Wisbech', 'land at Britannia Road, Stockton-On-Tees', 'Land at Great Wood Skirmett, Henley-On-Thames', 'land at Maltby Court, Oldham', 'land lying to the West of Windermere Road, Linthorpe', 'Land forming part of 7 Blenavon Court, Yarm', 'Land at Portrack, Stockton-On-Tees', 'Land adjoining River Stour, Part Of Hands Farm, Newbold On Stour, Stratford-Upon-Avon']

Summary:
Total missing LSOA: 8230
Contains keywords: 4977
Percentage: 60.5%


# do some classification

In [17]:
voa_businesses = load_voa_ratinglist(str(voa_path), postcode_district_lookup)

Initial rows: 2293839


### New classification

create VOA business gazetteer

In [19]:
with_matches = add_business_matches(enhanced, voa_businesses)

In [50]:
classified = property_class(with_matches)

classified.groupby('class').size()

class
airspace        1719
business       16768
carpark         2996
land           18228
residential    68335
unknown        11787
dtype: int64

In [22]:
classified.loc[classified['class']=='unknown', ['title_number','building_name','property_address','street_name2', 'street_number',
'building_match','street_match', 'number_match','class']]

,title_number,building_name,property_address,street_name2,street_number,building_match,street_match,number_match,class
4,MAN300971,timber wharf,"Unit 111, Timber Wharf, Worsley Street, Manche...",worsleystreet,NaN,False,True,False,unknown
16,CB400630,NaN,"2-24 (even) Miller Way, 15-25 Hammonds Drive (...",fengate,NaN,False,True,False,unknown
76,MAN203328,southern tower,"Southern Tower, Bengal Mill, Bengal Street, Ma...",bengalstreet,NaN,False,True,False,unknown
77,MAN203328,bengal mill,"Southern Tower, Bengal Mill, Bengal Street, Ma...",bengalstreet,NaN,False,True,False,unknown
80,MAN204044,the vibe,"Unit 3, The Vibe, 175 Broughton Lane, Salford ...",broughtonlane,NaN,False,True,False,unknown
...,...,...,...,...,...,...,...,...,...
119814,AGL355237,NaN,"Level 10 West, 5 Broadgate, London (EC2M 2QS)",broadgate,NaN,False,True,False,unknown
119827,AGL465195,ormond house,"4th Floor, Ormond House, 63 Queen Victoria Str...",queenvictoriastreet,NaN,False,True,False,unknown
119828,AGL470128,NaN,"2nd Floor, 52 Lime Street, London (EC3M 7AW)",limestreet,NaN,False,True,False,unknown
119830,AGL473595,NaN,"9th and 10th floors, 52 Lime Street, London (E...",limestreet,NaN,False,True,False,unknown


In [23]:
classified

,title_number,tenure,unit_id,unit_type,number_filter,building_name,street_number,street_name,postcode,city,...,lsoa11cd,msoa11cd,street_number2,street_name2,match_prob,geog_match,building_match,street_match,number_match,class
0,MAN296957,Leasehold,NaN,NaN,NaN,westleigh lodge care home,NaN,Nel Pan Lane,WN7 5JT,Leigh,...,E01006282,E02001313,NaN,nelpanlane,NaN,NaN,False,True,False,business
1,MAN297185,Leasehold,1,Flat,NaN,NaN,NaN,Canal Street,M1 3HE,Manchester,...,E01033654,E02006912,NaN,canalstreet,NaN,NaN,False,True,False,residential
2,MAN299183,Leasehold,201,Flat,NaN,NaN,NaN,Regent Road,M3 4AY,Manchester,...,E01033673,E02006916,NaN,regentroad,NaN,NaN,False,False,False,residential
3,MAN300821,Freehold,NaN,NaN,NaN,NaN,NaN,Gerard Street,WN4 9AA,Wigan,...,E01006223,E02001322,NaN,gerardstreet,NaN,NaN,False,True,False,land
4,MAN300971,Leasehold,111,Unit,NaN,timber wharf,NaN,Worsley Street,M15 4NZ,Manchester,...,E01033673,E02006916,NaN,worsleystreet,NaN,NaN,False,True,False,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119828,AGL470128,Leasehold,NaN,NaN,NaN,NaN,NaN,Lime Street,EC3M 7AW,London,...,E01032739,E02000001,NaN,limestreet,NaN,NaN,False,True,False,unknown
119829,AGL473126,Leasehold,NaN,NaN,NaN,NaN,155,Bishopsgate,EC2M 3AD,London,...,E01032739,E02000001,155,bishopsgate,NaN,NaN,False,True,True,business
119830,AGL473595,Leasehold,NaN,NaN,NaN,NaN,NaN,Lime Street,EC3M 7AF,London,...,E01032739,E02000001,NaN,limestreet,NaN,NaN,False,True,False,unknown
119831,AGL475468,Leasehold,NaN,NaN,NaN,NaN,NaN,Bevis Marks,EC3A 7BA,London,...,E01032739,E02000001,NaN,bevimarks,NaN,NaN,False,True,False,unknown


# Classify as multi property and expand

In [53]:

multi_tagged = tag_multi_property(df)

unit_expanded = expand_dataframe_numbers(
    df.loc[df['unit_id'].notna() & df['is_multi']].reset_index(drop=True), 
    column_name='unit_id', 
    print_every=10000, 
    min_count=1
)

street_expanded = expand_dataframe_numbers(
    df.loc[df['unit_id'].isna() & df['is_multi']].reset_index(drop=True), 
    column_name='street_number', 
    print_every=10000, 
    min_count=1
)

In [60]:
expanded_df = pd.concat([df.loc[~df['is_multi']], unit_expanded, street_expanded], ignore_index = True)



In [63]:
expanded_df['multi_id'] = expanded_df.groupby('title_number').cumcount() +1
expanded_df['unique_id'] = [
        str(x) + "-" + str(y)
        for x, y in zip(
            expanded_df["title_number"], expanded_df["multi_id"]
        )
    ]

In [64]:
expanded_df

,title_number,tenure,unit_id,unit_type,number_filter,building_name,street_number,street_name,postcode,city,...,street_name2,match_prob,geog_match,building_match,street_match,number_match,class,is_multi,multi_id,unique_id
0,MAN296957,Leasehold,NaN,NaN,none,westleigh lodge care home,NaN,Nel Pan Lane,WN7 5JT,Leigh,...,nelpanlane,NaN,NaN,False,True,False,business,False,1,MAN296957-1
1,MAN297185,Leasehold,1,Flat,none,NaN,NaN,Canal Street,M1 3HE,Manchester,...,canalstreet,NaN,NaN,False,True,False,residential,False,1,MAN297185-1
2,MAN299183,Leasehold,201,Flat,none,NaN,NaN,Regent Road,M3 4AY,Manchester,...,regentroad,NaN,NaN,False,False,False,residential,False,1,MAN299183-1
3,MAN300821,Freehold,NaN,NaN,none,NaN,NaN,Gerard Street,WN4 9AA,Wigan,...,gerardstreet,NaN,NaN,False,True,False,land,False,1,MAN300821-1
4,MAN300971,Leasehold,111,Unit,none,timber wharf,NaN,Worsley Street,M15 4NZ,Manchester,...,worsleystreet,NaN,NaN,False,True,False,unknown,False,1,MAN300971-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195045,SYK618891,Freehold,NaN,NaN,none,NaN,46,Windermere Drive,DN4 5PZ,Doncaster,...,windermeredrive,NaN,NaN,False,False,False,residential,True,34,SYK618891-34
195046,SYK618891,Freehold,NaN,NaN,none,NaN,47,Windermere Drive,DN4 5PZ,Doncaster,...,windermeredrive,NaN,NaN,False,False,False,residential,True,35,SYK618891-35
195047,SYK618891,Freehold,NaN,NaN,none,NaN,48,Windermere Drive,DN4 5PZ,Doncaster,...,windermeredrive,NaN,NaN,False,False,False,residential,True,36,SYK618891-36
195048,SYK618891,Freehold,NaN,NaN,none,NaN,49,Windermere Drive,DN4 5PZ,Doncaster,...,windermeredrive,NaN,NaN,False,False,False,residential,True,37,SYK618891-37
